# Welcome to the qRBM demo!
In this demo we will go over a simple use case for an RBM that is trained semi-classically, semi-quantumly. To get started, we import our dependencies, we will need pyquil, grove, numpy and scipy. 

## Creating the underlying distribution
We flip an unbiased coin to generate a sequence of random bits, this will be the underlying hidden distribution of our data.

In [4]:
from pyquil.quil import Program
import pyquil.api as api
from pyquil.gates import *

In [7]:
qvm = get_qc('9q-square-qvm')
p = Program()
p.inst(H(0), CNOT(0, 1))
r = qRBM(qvm, num_visible=4, num_hidden=1, n_quantum_measurements=None, verbose=True)


NameError: name 'qRBM' is not defined

In [1]:
from pyquil.api import local_forest_runtime
from pyquil import get_qc, Program

In [9]:
import pyquil.api as api
qvm_connection = api.QVMConnection()

In [16]:
qc = get_qc('9q-square-qvm')

In [25]:
#Get our imports and setup out of the way...
from pyquil.api import local_forest_runtime
from pyquil import get_qc, Program
import pyquil.api as api
from random import *
import numpy as np
from qRBM_final import qRBM
# qvm = get_qc('9q-square-qvm')
qvm = api.QVMConnection()


#flip a coin
np.random.seed(1234)
random_coin = np.random.choice([-1,1], size=20, replace=True)
random_coin

array([ 1,  1, -1,  1, -1, -1, -1,  1,  1,  1,  1,  1, -1, -1,  1, -1, -1,
       -1, -1, -1])

## Generating encoded sequences
Code below takes our sequence of random bits and encodes each 1 bit result into a 4 bit representation. This effectively creates artificial data with a 1 bit "needle" (hidden code subspace) in the "haystack" (4-bit data), which our RBM will have to decode. 

In [22]:
#Encode this coin flip in to an artifically high dimensional dataset
artificial_data = []
for flip in random_coin:
    if flip == 1:
        artificial_data.append([1,1,-1,-1]) #logical 1
    else:
        artificial_data.append([-1,-1,1,1]) #logical 0

#We know have an artificially high dimensional dataset that still only has the 1 bit coin flip dictating
#the underlying distribution
artificial_data = np.asarray(artificial_data)
artificial_data

array([[ 1,  1, -1, -1],
       [ 1,  1, -1, -1],
       [-1, -1,  1,  1],
       [ 1,  1, -1, -1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1],
       [ 1,  1, -1, -1],
       [ 1,  1, -1, -1],
       [ 1,  1, -1, -1],
       [ 1,  1, -1, -1],
       [ 1,  1, -1, -1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1],
       [ 1,  1, -1, -1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1]])

## Creating and training our qRBM

In [26]:
#We will now setup our RBM to try and understand the artifical data's underlying distribution

#we will use "analytical" measurement to save time on simulation.
qr = qRBM(qvm, num_visible=4, num_hidden=1,n_quantum_measurements=None)

In [27]:
%%capture
#let it train...
qr.train(artificial_data)

## Comparing RBM-decoded data to originally encoded data
In the following code snippet we examine the hidden unit activation probabilities and how they correspond to the initial information from the coin flip. As we can see the probability of the hiddens corresponds very well with the flip.

In [29]:
#Now that the training is done (~5 mins when doing analytical expectation)
# we can transform our data to the hidden layer
transformed = qr.transform(artificial_data)

comparison = np.stack((transformed[:,0], random_coin))

#compare our rbm probabilities with the coin flips.
print ('RBM Pr. | Original Coin Value')
print ('-'*20)
for i in range(len(transformed)):
    print ('{: 0.3f}'.format(float(comparison[:,i][0])), ' |', '{: 0.3f}'.format(comparison[:,i][1]))


RBM Pr. | Original Coin Value
--------------------
 0.000  |  1.000
 0.000  |  1.000
 1.000  | -1.000
 0.000  |  1.000
 1.000  | -1.000
 1.000  | -1.000
 1.000  | -1.000
 0.000  |  1.000
 0.000  |  1.000
 0.000  |  1.000
 0.000  |  1.000
 0.000  |  1.000
 1.000  | -1.000
 1.000  | -1.000
 0.000  |  1.000
 1.000  | -1.000
 1.000  | -1.000
 1.000  | -1.000
 1.000  | -1.000
 1.000  | -1.000


# Thank you!
Check out 
https://github.com/MichaelBroughton/QABoM.
For a baseline version of this code that you can experiment with for yourself.